In [1]:
using SparseArrays
using LinearAlgebra
using NLopt
using PyPlot
using KrylovKit
using FiniteDifferences
using Revise
using TopologyOptimizationHelper

Precompiling TopologyOptimizationHelper
  ✓ TopologyOptimizationHelper
  1 dependency successfully precompiled in 7 seconds. 39 already precompiled.


In [2]:
# Our favorite parameters for testing
# Re-run to randomize
L = 20
ε = rand(480) .* 11 .+ 1
δε = randn(length(ε)) * 1e-5
ω = 2π
δω = randn() * 1e-5

A, x = Maxwell1d(L, ε, ω)
new_ε_A, _ = Maxwell1d(L, ε + δε, ω)
new_ω_A, _ = Maxwell1d(L, ε, ω + δω; ω_pml = ω)

M = length(x)
b = zeros(M)
b[M÷2] = 1;

In [9]:
# Exact gradient of LDOS versus numerical gradient
LDOS, ∇LDOS = ∇_ε_LDOS(A, ω, b)
new_ε_LDOS, _ = ∇_ε_LDOS(new_ε_A, ω, b)

@show new_ε_LDOS - LDOS
∇LDOS' * δε

new_ε_LDOS - LDOS = 1.3741776631441357e-10


1.3741223261937758e-10

In [4]:
# Exact gradient of eigenvalue versus numerical gradient
val, gradient = Eigengradient(A, ε, ω, b)
test_val, _ = Arnoldi_eig(A, ε, ω, b)
new_val, _ = Eigengradient(new_ε_A, ε + δε, ω, b)
@show real(new_val) - real(val)
@show dot(δε, real(gradient))

# Exact gradient of eigenvalue versus Richardson extrapolation
dir = randn(M)
f = α -> Maxwell_omega(L, ε + α * dir, ω, b)
@show extrapolate_fdm(central_fdm(2, 1), f, 0)
dir' * gradient

real(new_val) - real(val) = -2.4064662884626387e-7
dot(δε, real(gradient)) = -2.406477312601946e-7
extrapolate_fdm(central_fdm(2, 1), f, 0) = (-0.03328211831066301 + 5.801695187823441e-5im, 1.288834395300415e-13)


-0.033282118311763295 + 5.801695187374557e-5im

In [10]:
# Exact gradient of LDOS versus Richardson extrapolation
∇LDOS = ∇_ω_LDOS(A, ε, ω, b)
new_ω_LDOS, _ = ∇_ε_LDOS(new_ω_A, ω, b)
@show new_ω_LDOS - LDOS
∇LDOS[1]' * real(δω)

new_ω_LDOS - LDOS = 5.516575073435933e-9


5.506296334671343e-9

In [14]:
ω₀, ∂ω_∂ε = Eigengradient(A, ε, ω, b)
A₀, _ = Maxwell1d(L, ε, real(ω₀); ω_pml=ω)
true_LDOS, ∂LDOS_∂ε = ∇_ε_LDOS(A₀, real(ω₀), b)
∂LDOS_∂ω = ∇_ω_LDOS(A₀, ε, real(ω₀), b)

new_ω₀ = Maxwell_omega(L, ε + δε, ω, b)
new_A₀, _ = Maxwell1d(L, ε + δε, real(new_ω₀); ω_pml=ω)
true_new_LDOS = Just_Improved_LDOS(L, ε + δε, ω, b)

true_∇LDOS = ∂LDOS_∂ε .+  ∂LDOS_∂ω .* real.(∂ω_∂ε)

@show true_new_LDOS - true_LDOS
true_∇LDOS' * δε

true_new_LDOS - true_LDOS = -2.1443974608514882e-7


-2.144381386476221e-7

In [15]:
dir = randn(length(ε))

f = α -> Just_Improved_LDOS(L, ε + α * dir, ω, b)
@show extrapolate_fdm(central_fdm(2, 1), f, 0)
dir' * true_∇LDOS

extrapolate_fdm(central_fdm(2, 1), f, 0) = (-0.02705025981770696, 1.5797953223373185e-12)


-0.027050259817991716

In [19]:
using SparseArrays
using LinearAlgebra
using NLopt
using PyPlot
using KrylovKit
using FiniteDifferences
using TopologyOptimizationHelper
using Test

@testset "TopologyOptimizationHelper.jl" begin
    # Initialize parameters for testing
    L = 20
    ε = rand(480) .* 11 .+ 1
    δε = randn(length(ε)) * 1e-6
    ω = 2π
    δω = randn() * 1e-6
    dir = randn(length(ε))

    A, x = Maxwell1d(L, ε, ω)
    new_ε_A, _ = Maxwell1d(L, ε + δε, ω)
    new_ω_A, _ = Maxwell1d(L, ε, ω + δω; ω_pml = ω)

    M = length(x)
    b = zeros(M)
    b[M÷2] = 1


    # Gradient of LDOS w.r.t. ε
    ## Exact gradient versus numerical gradient
    LDOS, ∇LDOS = ∇_ε_LDOS(A, ω, b)
    
    new_ε_LDOS, _ = ∇_ε_LDOS(new_ε_A, ω, b)
    @test new_ε_LDOS - LDOS ≈ ∇LDOS' * δε atol=1e-10


    # Gradient of eigenvalue w.r.t. ε
    ## Exact gradient versus numerical gradient
    val, _ = Arnoldi_eig(A, ε, ω, b)
    gradient = Eigengradient(A, ε, ω, b)
    
    new_val, _ = Arnoldi_eig(new_ε_A, ε + δε, ω, b)
    @test sqrt(new_val) - sqrt(val) ≈ dot(δε, gradient) atol=1e-10

    ## Exact gradient versus Richardson extrapolation
    f = α -> Maxwell_omega(L, ε + α * dir, ω, b)
    @test extrapolate_fdm(central_fdm(2, 1), f, 0)[1] ≈ dir' * gradient atol=1e-10


    # Gradient of LDOS w.r.t. ω 
    ## Exact gradient versus numerical gradient
    ∇LDOS = ∇_ω_LDOS(A, ε, ω, b)
    
    new_ω_LDOS, _ = ∇_ε_LDOS(new_ω_A, ω, b)
    @test new_ω_LDOS - LDOS ≈ ∇LDOS[1] * real(δω) atol=1e-10


    # Improved gradient of LDOS w.r.t. ε
    ## Exact gradient versus numerical gradient
    ω₀ = Maxwell_omega(L, ε, ω, b)
    A₀, _ = Maxwell1d(L, ε, real(ω₀); ω_pml=ω)
    true_LDOS, ∂LDOS_∂ε = ∇_ε_LDOS(A₀, real(ω₀), b)
    ∂LDOS_∂ω = ∇_ω_LDOS(A₀, ε, real(ω₀), b)
    ∂ω_∂ε = real.(Eigengradient(A, ε, ω, b))
    true_∇LDOS = ∂LDOS_∂ε .+  ∂LDOS_∂ω .* ∂ω_∂ε
    
    new_ω₀ = Maxwell_omega(L, ε + δε, ω, b)
    new_A₀, _ = Maxwell1d(L, ε + δε, real(new_ω₀); ω_pml=ω)
    true_new_LDOS = Just_Improved_LDOS(L, ε + δε, ω, b)
    @test true_new_LDOS - true_LDOS ≈ true_∇LDOS' * δε atol=1e-10

    ## Exact gradient versus Richardson extrapolation
    f = α -> Just_Improved_LDOS(L, ε + α * dir, ω, b)
    @test extrapolate_fdm(central_fdm(2, 1), f, 0)[1] ≈ dir' * true_∇LDOS atol=1e-10

end

TopologyOptimizationHelper.jl: Error During Test at In[19]:42
  Test threw exception
  Expression: ≈(sqrt(new_val) - sqrt(val), dot(δε, gradient), atol = 1.0e-10)
  DimensionMismatch: x and y are of different lengths!
  Stacktrace:
   [1] dot
     @ C:\Users\jorge\.julia\juliaup\julia-1.10.4+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\generic.jl:871 [inlined]
   [2] dot(x::Vector{Float64}, y::Tuple{ComplexF64, Vector{ComplexF64}})
     @ LinearAlgebra C:\Users\jorge\.julia\juliaup\julia-1.10.4+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\generic.jl:868
   [3] macro expansion
     @ C:\Users\jorge\.julia\juliaup\julia-1.10.4+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Test\src\Test.jl:669 [inlined]
   [4] macro expansion
     @ In[19]:42 [inlined]
   [5] macro expansion
     @ C:\Users\jorge\.julia\juliaup\julia-1.10.4+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Test\src\Test.jl:1577 [inlined]
   [6] top-level scope
     @ In[19]:12
TopologyOptimizationHel

LoadError: [91mSome tests did not pass: 2 passed, 0 failed, 3 errored, 0 broken.[39m